# Train and evaluate model

This notebook recreates Table 1 from the manuscript. We train a basic CNN encoder from scratch on a dataset of sparse polynomials. We then evaluate reconstruction performance on a set of basis systems from across the sciences. 

Our coding framework is based on a Click interface and we make use of that in this notebook by running the basic steps in the pipeline through shell commands. 


In [ ]:
import os
import subprocess
from phase2vec.utils import get_command_defaults, ensure_dir, write_yaml, update_yaml
from phase2vec.train import load_model, train_model, run_epoch
from phase2vec.data import load_dataset, SystemFamily

## Generate data
First, we generate both the training and testing sets. The former will be a set of vector fields corresponding to polynomial ODEs of degree at most 3 and having sparse coefficients. The testing set will be vector fields representing the flows of 10 types of sytsems drawn from across the sciences. In all cases, we work with planar (i.e. two-dimensional systems). 

First, we set some basic parameters, including the types of training and testing data and the number of their samples. 

In [ ]:
## Generate data

data_dir = os.path.join(os.path.expanduser('~'),'data/phase2vec') # Alter to change where all of the phase2vec data will be saved. 

# Edit the data included in training and testing here. 
train_data_names = ['polynomial']
test_data_names  = ['saddle_node', 'pitchfork', 'transcritical',
                    'selkov', 'homoclinic', 'vanderpol',
                    'simple_oscillator', 'fitzhugh_nagumo', 'lotka_volterra']

# test_data_names  = ['lotka_volterra']
train_system_classes = []
test_system_classes = []
for n, names in enumerate([train_data_names, test_data_names]):
    for system in [SystemFamily(data_name=name) for name in names]:
        if n == 0:
            train_system_classes += [system.data_name + ' ' + str(i) for i in range(len(system.param_groups))]
        else:
            test_system_classes += [system.data_name + ' ' + str(i) for i in range(len(system.param_groups))]

num_train_classes = len(train_system_classes)
num_test_classes = len(test_system_classes)

# Edit the number of total samples from each data set here.
#By default, each set is divied further into a base and validation set at a 75/100 split. This can be altered below. 
num_train_samples = 10000 # total number of train/val samples
num_test_samples  = 2000 # total number of test samples. Note these are split themselves automatically into a regular and a validation component, but they can be combined. 
device            = 'cpu' # set to `cpu` if cuda not available

# Leave this untouched unless you want to change how parameters from each system are sampled and the proportions of each system in the data set.
test_samplers    = ['uniform'] * len(test_data_names)
test_props       = [str(1. / len(test_data_names))] * len(test_data_names)
test_data_names   = '-s ' +  ' -s '.join(test_data_names)
test_samplers     = '-sp ' +  ' -sp '.join(test_samplers)
test_props = '-c ' +  ' -c '.join(test_props)

Next, we call the actual shell commands for generating the data. These commands will make two directories, called `polynomial` and `classical`, corresponding to train and test sets, inside your `data_dir`. 

In order to alter the validation proportion, $p$, add the flag `--val-size <p>` where $p\in (0,1)$. 

In [ ]:
subprocess.call(f'phase2vec generate-dataset --data-dir {data_dir} --data-set-name classical --num-samples {num_test_samples} {test_data_names} {test_samplers} {test_props}', shell=True)

For the training data, we make sure to include the path to the forms for the testing data so that forms are not duplicated.

In [ ]:
holdout_fn = os.path.join(data_dir, 'classical', 'forms.npy')
subprocess.call(f'phase2vec generate-dataset --data-dir {data_dir} --num-samples {num_train_samples} --data-set-name polynomial --system-names {train_data_names[0]} -sp control -h {holdout_fn}', shell=True)

## Instantiate `phase2vec` encoder. 

We build the embedding CNN. We use the default parameters which we access by fetching the default arguments from the click command `generate_net_config`. To edit these parameters, alter the values of the dictionary `net_info`. 

* **model_type** (str): which of the pre-built architectures from _models.py to load. Make your own by combining modules from _modules.py 
* **latent_dim** (int): embedding dimension
* Continue...

In [ ]:
## Set net parameters
from phase2vec.cli._cli import generate_net_config

net_info = get_command_defaults(generate_net_config)
model_type = net_info['net_class']

# These parameters are not considered architectural parameters for the net, so we delete them before they're passed to the net builder. 
del net_info['net_class']
del net_info['output_file']
del net_info['pretrained_path']
del net_info['ae']

net = load_model(model_type, pretrained_path=None, device=device, **net_info).to(device)

## Set training parameters and load data. 

Next, we set the optimization parameters for training. As before, we fetch the default arguments from the relevant click command, `call_train`. These parameters can be updated by altering the values of the dictionary `train_info`. 

In [ ]:
## Set training parameters
from phase2vec.cli._cli import call_train

train_info = get_command_defaults(call_train)
train_info['log_dir']    = os.path.join(os.path.expanduser('~'),'runs')
train_info['num_epochs'] = 100
beta = 1e-3
train_info['beta'] = beta
train_info['exp_name']   = f'sparse_train_{beta}'

# These are only used by the click interface. 
del train_info['model_save_dir']
del train_info['seed']
del train_info['config_file']

# Set some training paths

pretrained_path = None # Replace with model_save_dir in order to load a pretrained model
model_save_dir  = os.path.join(os.path.expanduser('~'), 'phase2vec', train_info['exp_name'])
ensure_dir(model_save_dir)
ensure_dir(train_info['log_dir'])

# Where is training data stored? 
train_data_path = os.path.join(data_dir, 'polynomial')

# Load training data. 
X_train, X_val, y_train, y_val, p_train, p_val = load_dataset(train_data_path)

Now, we actually train the model. By default, you can observe training at http://localhost:6007/ and TensorBoard summaries are saved in `train_info['logdir']`. 

In [ ]:
# Train the model
log_dir = train_info['log_dir']
subprocess.call(f'rm -rf {log_dir}/* &', shell=True)
subprocess.call(f'tensorboard --logdir {log_dir}&', shell=True)
net = train_model(X_train, X_val,
                  y_train, y_val,
                  p_train, p_val,
                  net,**train_info)

# Save it
from torch import save
save(net.state_dict(), os.path.join(model_save_dir, 'model.pt'))

## Evaluation

We evaluate the model and compare it to a per-equation LASSO baseline. First, we load the testing data (putting it all into one big data set) and make sure that function forms between train and test are not duplicated. 

In [ ]:
import numpy as np
import pdb

results_dir = os.path.join(os.path.expanduser('~'),f'results/phase2vec/{train_info["exp_name"]}')
ensure_dir(results_dir)
# Load testing data

test_data_path = os.path.join(data_dir, 'classical')
X_test1, X_test2, y_test1, y_test2, p_test1, p_test2 = load_dataset(test_data_path)
X_test = np.concatenate([X_test1, X_test2])
y_test = np.concatenate([y_test1, y_test2])
p_test = np.concatenate([p_test1, p_test2])

# Quickly check for bad forms
forms_train = 1 * (p_train != 0)
forms_test = np.load(holdout_fn)

counter = 0
for ftr in forms_train:
    bad_form = np.any(np.all(ftr == forms_test))
    if bad_form:
        counter+=1
print(f'{counter} bad forms detected')   

We write here a couple of helper functions that make it easy to compare `phase2vec` to `LASSO` in parallel. 

In [ ]:
def forward_p2v(net, data, **kwargs):
    b = data.shape[0]
    n = data.shape[2]
    emb  = net.emb(net.enc(data).reshape(b, -1))
    out  = net.dec(emb)
    pars = out.reshape(-1,net.library.shape[-1], 2)
    recon = torch.einsum('sl,bld->bsd',net.library.to(device),pars).reshape(b, n,n,2).permute(0,3,1,2)
    return pars, recon

def forward_lasso(net, data, **kwargs):
    b = data.shape[0]
    n = data.shape[2]
    alpha = kwargs['beta']
    # LASSO
    pars = []
    for z in data:
        zx = z[0,:,:].numpy().flatten()
        zy = z[1,:,:].numpy().flatten()
        clf = linear_model.Lasso(alpha=alpha)
        clf.fit(net.library.numpy(), zx)
        mx = clf.coef_
        clf.fit(net.library.numpy(), zy)
        my = clf.coef_
        pars.append(torch.stack([torch.tensor(mx), torch.tensor(my)]))
    pars = torch.stack(pars).permute(0,2,1)
    recon = torch.einsum('sl,bld->bsd',net.library.to(device),pars).reshape(b,n,n,2).permute(0,3,1,2)  
    return pars, recon

Finally, we evaluate on a per class basis. 

In [ ]:
# Evaluate
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import linear_model
from phase2vec.train._losses import *
loss_dict = {}
sorted_data = []
recon_dict = {'p2v':[],'lasso':[]}
fp_normalize = True

euclidean = normalized_euclidean if fp_normalize else euclidean_loss

# Don't forget to do this. 
net.eval()
for label in tqdm(np.unique(y_test)):

    # Data for this class
    data   = torch.tensor([datum for (d, datum) in enumerate(X_test) if y_test[d] == label])
    pars   = torch.tensor([par for (p, par) in enumerate(p_test) if y_test[p] == label])
    labels = torch.ones(len(data)) * label
    
    sorted_data += list(data)
    
    # Both the p2v and lasso loss for this class
    class_par_losses = []
    class_recon_losses = []
    # For each fitting method
    for nm, forward in zip(recon_dict.keys(),[forward_p2v, forward_lasso]):
        
        # Fit pars and return recon
        pars_fit, recon = forward(net, data.float(), beta=beta)
    
        # Par loss
        par_loss   = euclidean_loss(pars_fit, pars).detach().cpu().numpy()
        # Recon loss
        recon_loss = euclidean(recon, data).detach().cpu().numpy()
        class_par_losses.append(par_loss)
        class_recon_losses.append(recon_loss) 
        
        recon_dict[nm] += list(recon)    
    loss_dict[test_system_classes[label]] = class_recon_losses + class_par_losses
df = pd.DataFrame(data=loss_dict)
df.index = [ 'Recon-P2V', 'Recon-LASSO', 'Param-P2V', 'Param-LASSO']

In [ ]:
# Show data frame of results
df

In [ ]:
# Show average performance of phase2vec vs lasso
df.mean(axis=1)